In [44]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [45]:
from classes.grid import Grid

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [46]:
class Warehouse(Grid):
    def __init__(self, grid):
        super().__init__(grid)
        self.robot_pos = self.get_tiles('@').pop()

    def apply_robot_instruction(self, dir):
        '''
        Moves the robot (and potentially some boxes) once in the direction dir
        '''
        match dir:
            case '^':
                dir_format = 'up'
            case '<':
                dir_format = 'left'
            case '>':
                dir_format = 'right'
            case 'v':
                dir_format = 'down'

        # Find how far away the next wall is in this direction, and how long the immediate chain of boxes is
        wall_steps_away = 1
        box_chain_on = True
        boxes_directly_in_front = 0

        while (True):
            # What tile is [iteration x] steps away in this direction?
            look_ahead = self.get_relative(self.robot_pos[0],self.robot_pos[1],dir_format,step=wall_steps_away)

            # Track how many boxes are directly in front
            if look_ahead[2] == 'O' and box_chain_on:
                boxes_directly_in_front += 1
            else:
                box_chain_on = False
            
            # Stop once we hit a wall
            if look_ahead[2] == '#':
                break
            wall_steps_away += 1

        # Look ahead in this direction up to the wall, and push all concurrent tiles
        # before a . up one square, and replace the existing tiles with the one that is one place before
        
        # Skip if we are squished up against a wall
        if boxes_directly_in_front == wall_steps_away-1:
            return


        # Loop through how long the current chain of boxes is (go in reverse to preserve grid info)
        for tile_shift in range(boxes_directly_in_front, -1, -1):

            # Move each box up one position in the direction that we're moving
            match dir:
                case '^':
                    self.grid[self.robot_pos[0]-tile_shift-1][self.robot_pos[1]] = self.grid[self.robot_pos[0]-tile_shift][self.robot_pos[1]]
                case 'v':
                    self.grid[self.robot_pos[0]+tile_shift+1][self.robot_pos[1]] = self.grid[self.robot_pos[0]+tile_shift][self.robot_pos[1]]
                case '<':
                    self.grid[self.robot_pos[0]][self.robot_pos[1]-tile_shift-1] = self.grid[self.robot_pos[0]][self.robot_pos[1]-tile_shift]
                case '>':
                    self.grid[self.robot_pos[0]][self.robot_pos[1]+tile_shift+1] = self.grid[self.robot_pos[0]][self.robot_pos[1]+tile_shift]
            
            # If we've reached the end of the chain of consecutitve boxes, then we have reached the robot,
            # so we update its position
            if tile_shift == 0:
                self.grid[self.robot_pos[0]][self.robot_pos[1]] = '.'
                self.robot_pos = self.get_tiles('@').pop()
                break
    
    def gps_sum(self):
        '''
        Returns the sum of all boxes' GPS coordinates of the current grid
        '''
        boxes = self.get_tiles('O')

        total_gps_sum = 0
        for box in boxes:
            total_gps_sum += box[0]*100 + box[1]
            
        return total_gps_sum


In [47]:
warehouse_info = input.split('\n\n')

warehouse_map_input = warehouse_info[0].split('\n')
robot_directions = ''.join(warehouse_info[1].split('\n'))

warehouse_map = Warehouse(warehouse_map_input)
#warehouse_map.print_grid()

for dir in robot_directions:
    #print(f'moving {dir}')
    warehouse_map.apply_robot_instruction(dir)
#warehouse_map.print_grid()
warehouse_map.gps_sum()

1514333

# Part 2

In [48]:
class WideWarehouse(Grid):
    def __init__(self, grid):
        super().__init__(grid)

        # Will track the boxes as objects this time around
        self.boxes = set()
        self.box_coords = {}
        
        # Need to re-initialise the grid in part 2 to widen everything
        new_grid = [['.']*self.num_cols*2 for _ in range(self.num_rows)]

        for row in range(self.num_rows):
            for col in range(self.num_cols):
                match self.grid[row][col]:
                    case '#':
                        new_grid[row][col*2] = '#'
                        new_grid[row][col*2+1] = '#'
                    case '.':
                        new_grid[row][col*2] = '.'
                        new_grid[row][col*2+1] = '.'
                    case 'O':
                        new_grid[row][col*2] = '['
                        new_grid[row][col*2+1] = ']'

                        new_box = Box((row,col*2), (row,col*2+1), self)
                        self.boxes.add(new_box)
                        self.box_coords[new_box.left_pos] = new_box
                        self.box_coords[new_box.right_pos] = new_box
                    case '@':
                        new_grid[row][col*2] = '@'
                        new_grid[row][col*2+1] = '.'
        
        self.grid = new_grid
        self.num_cols *= 2
        self.robot_pos = self.get_tiles('@').pop()
    
    def apply_instruction(self,dir):
        '''
        Applies direction function to robot and potentially pushes some boxes around
        '''
        next_tile = self.get_relative(self.robot_pos[0],self.robot_pos[1],dir)

        match next_tile[2]:
            case '#':
                return
            case '[' | ']':
                adj_box = self.box_coords[(next_tile[0],next_tile[1])]
                boxes_to_move = adj_box.get_all_connected_boxes(dir)

                if not boxes_to_move:
                    return

                for box in boxes_to_move:
                    box.move_box(dir)
        
        # Finally, if there's room to move we update robot's position
        self.grid[self.robot_pos[0]][self.robot_pos[1]] = '.'

        match dir:
            case '^':
                self.robot_pos = (self.robot_pos[0]-1, self.robot_pos[1])
            case 'v':
                self.robot_pos = (self.robot_pos[0]+1, self.robot_pos[1])
            case '<':
                self.robot_pos = (self.robot_pos[0], self.robot_pos[1]-1)
            case '>':
                self.robot_pos = (self.robot_pos[0], self.robot_pos[1]+1)

        self.grid[self.robot_pos[0]][self.robot_pos[1]] = '@'

    def gps_sum(self):
        '''
        Returns the sum of all boxes' GPS coordinates of the current grid
        '''
        boxes = self.get_tiles('[')

        total_gps_sum = 0
        for box in boxes:
            total_gps_sum += box[0]*100 + box[1]
            
        return total_gps_sum


class Box():
    def __init__(self, left_pos, right_pos, wide_warehouse: WideWarehouse):
        self.left_pos = left_pos
        self.right_pos = right_pos
        self.wide_warehouse = wide_warehouse
    
    def move_box(self, dir):
        '''
        Moves this box object one space in its grid in the given direction
        and updates this movement on the grid
        '''
        #print(f'moving box at {(self.left_pos,self.right_pos)}')

        self.wide_warehouse.grid[self.left_pos[0]][self.left_pos[1]] = '.'
        self.wide_warehouse.grid[self.right_pos[0]][self.right_pos[1]] = '.'
        self.wide_warehouse.box_coords.pop(self.left_pos)
        self.wide_warehouse.box_coords.pop(self.right_pos)

        match dir:
            case '^':
                self.left_pos = (self.left_pos[0]-1, self.left_pos[1])
                self.right_pos = (self.right_pos[0]-1, self.right_pos[1])
            case 'v':
                self.left_pos = (self.left_pos[0]+1, self.left_pos[1])
                self.right_pos = (self.right_pos[0]+1, self.right_pos[1])
            case '<':
                self.left_pos = (self.left_pos[0], self.left_pos[1]-1)
                self.right_pos = (self.right_pos[0], self.right_pos[1]-1)
            case '>':
                self.left_pos = (self.left_pos[0], self.left_pos[1]+1)
                self.right_pos = (self.right_pos[0], self.right_pos[1]+1)

        self.wide_warehouse.grid[self.left_pos[0]][self.left_pos[1]] = '['
        self.wide_warehouse.grid[self.right_pos[0]][self.right_pos[1]] = ']'
        self.wide_warehouse.box_coords[self.left_pos] = self
        self.wide_warehouse.box_coords[self.right_pos] = self
    
    def get_all_connected_boxes(self, dir):
        '''
        Returns a list of all boxes that, if this box were to be pushed in the direction 'dir',
        would also be pushed successfully (i.e. no wall in the way). The list is in order of boxes
        furthest away from the robot first (with respect only to the axis being pushed)

        e.g.

        ......
        .[][].
        ..[]..
        ..@...

        For dir='^' (pushing up), the list is ordered such that the two 'top' boxes
        are first and the 'lower' box is the final entry
        '''

        connected_boxes = [self]

        # Edges that we still need to look at - initialise to the starting box
        match dir:
            case '^' | 'v':
                edges_to_check = [self.left_pos, self.right_pos]
            case '<':
                edges_to_check = [self.left_pos]
            case '>':
                edges_to_check = [self.right_pos]

        while(edges_to_check):
            cur_tile = edges_to_check.pop(0)

            # What is the next tile along?
            match dir:
                case '^':
                    next_tile_coords = (cur_tile[0]-1,cur_tile[1])
                case 'v':
                    next_tile_coords = (cur_tile[0]+1,cur_tile[1])
                case '>':
                    next_tile_coords = (cur_tile[0],cur_tile[1]+1)
                case '<':
                    next_tile_coords = (cur_tile[0],cur_tile[1]-1)
            
            # If we hit a wall, none of the boxes will be able to move in this direction so end early
            next_tile = self.wide_warehouse.grid[next_tile_coords[0]][next_tile_coords[1]]
            if next_tile == '#':
                return []

            # If we don't see an immediate wall yet and there's another box, check that box too
            if next_tile_coords in self.wide_warehouse.box_coords:
                next_box = self.wide_warehouse.box_coords[next_tile_coords]
                match dir:
                    case '^' | 'v':
                        edges_to_check.append(next_box.left_pos)
                        edges_to_check.append(next_box.right_pos)
                    case '<':
                        edges_to_check.append(next_box.left_pos)
                    case '>':
                        edges_to_check.append(next_box.right_pos)
                        
                if next_box not in connected_boxes:
                    connected_boxes.append(next_box)
        
        return reversed(connected_boxes)

In [49]:
warehouse_info = input.split('\n\n')

warehouse_map_input = warehouse_info[0].split('\n')
robot_directions = ''.join(warehouse_info[1].split('\n'))

warehouse = WideWarehouse(warehouse_map_input)
#warehouse.print_grid()

for direction in robot_directions:
    warehouse.apply_instruction(direction)
    #print(f'moving {direction}')
    #warehouse.print_grid()

warehouse.gps_sum()

1528453